In [ ]:
import enterprise
import enterprise.signals.parameter as parameter
import numpy as np
import matplotlib.pyplot as plt
import corner
import healpy as hp
import os, glob, json, pickle
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
from QuickBurst import enterprise_wavelets as models
from enterprise.signals.parameter import function
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import re
#style
import cProfile

In [ ]:
psrlist = ['J0030+0451', 'J0613-0200', 'J1455-3330', 'J1600-3053', 'J1614-2230', 'J1713+0747',
                  'J1744-1134', 'J1832-0836', 'J1853+1303', 'J1909-3744', 'J1918-0642', 'J2010-1323',
                  'J2317+1439', 'J1741+1351', 'J2043+1711']
parfiles = ['/home/reyna/Downloads/nb_tims/{0}_PINT_20210117.nb.par'.format(psrlist[k])]
timfiles = [ '/home/reyna/Downloads/nb_tims/{0}.nb.tim'.format(psrlist[k]) ]
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem='DE438', clk='TT(BIPM2018)',timing_package = 'PINT')
    psrs.append(psr)
#******************************************************************************************************************************#
def analysis(psrs):
    the_pulsar = psrlist[k]
    print ('The Pulsar Considered: {}'.format(the_pulsar))
    psr = psrs[k]

    # find the maximum time span to set red-noise/DM-variation frequency sampling for either pickled srialized or not:
    tmin = psr.toas.min()
    tmax = psr.toas.max()
    Tspan = np.max(tmax) - np.min(tmin)
    print('Tspan is: ', Tspan/(365*3600*24))
    # define selection by observing backend: make different white noises for each backend combo
    selection = selections.Selection(selections.by_backend)
    # special selection for ECORR only use wideband NANOGrav data
    selection2 = selections.Selection(selections.nanograv_backends)
    #.................................................Create Parameters................................................#
    # white noise parameters
    efac = parameter.Uniform(0.0, 10)
    equad = parameter.Uniform(-8.5, -5)
    ecorr = parameter.Uniform(-8.5, -5)
    # white noise
    ef = white_signals.MeasurementNoise(efac=efac, selection=selection)
    eq = white_signals.EquadNoise(log10_equad=equad, selection=selection)
    ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection2)
    # custom frequency vector:
    freqs= np.arange(1/Tspan, (bins + 0.001)/Tspan, 1/Tspan)
    # Path to be created
    path = parent_path + 'Desktop/15YearV0/Powerlaw/' + str(bins)+'_Freq/' + the_pulsar
    os.makedirs(path, exist_ok = True);
    # red noise parameters
    #log10_A = parameter.Uniform(-20, -11)
    #gamma = parameter.Uniform(0, 7)
    #pl = utils.powerlaw(log10_A=log10_A, gamma=gamma)
    #rn = gp_signals.FourierBasisGP(spectrum=pl, modes=freqs)

    '''

    #For the spectral model:
    #log10_rho = parameter.Uniform(-10, -4, size=len(freqs))
    #fs = gp_prior.free_spectrum(log10_rho=log10_rho)
    #rn = gp_signals.FourierBasisGP(spectrum=fs, modes=freqs)

    '''
    '''

    #enterprise_extensions version
    #wn = ee_models.white_noise_block(vary=True,inc_ecorr=True)
    #rn = ee_models.red_noise_block(psd='spectrum',modes=freqs)
    #pta = signal_base.PTA(model(psr))

    '''
    #.................................................Timing Model.................................................#
    tm = gp_signals.TimingModel(use_svd=True)
    #...................................................Signal.....................................................#
    #s = ef + eq + rn + tm
    s = ef + eq + tm
    #.................................................PTA Object...................................................#
    models = []
    if 'NANOGrav' in psr.flags['pta']:
        s2 = s + ec # ecorr only applied to NANOGrav data
        models.append(s2(psr))
    else:
        models.append(s(psr))
    pta = signal_base.PTA(models)
    #..................................Draw initial sample from model parameter space#.............................#
    x0 = np.hstack(p.sample() for p in pta.params)
    ndim = len(x0)
    #...............................................Setup sampler#.................................................#
    # initial jump covariance matrix
    cov = np.diag(np.ones(ndim) * 0.01**2) # helps to tune MCMC proposal distribution
    # where chains will be written to
    outdir = path.format(str(psr.name))
    # sampler object
    sampler = ptmcmc(ndim, pta.get_lnlikelihood, pta.get_lnprior, cov,
                     outDir=outdir,
                     resume=True)
    jp = samp.JumpProposal(pta)
    sampler.addProposalToCycle(jp.draw_from_prior, 15)
    sampler.addProposalToCycle(jp.draw_from_red_prior, 15)

    #np.savetxt(outdir+'/achrom_freqs.txt', freqs)
    np.savetxt(outdir+'/pars.txt',list(map(str, pta.param_names)), fmt='%s')
    np.savetxt(outdir+'/priors.txt',list(map(lambda x: str(x.__repr__()), pta.params)), fmt='%s')
    #........................................Sample the parameter space#.........................................#
    # sampler for N steps
    samp_steps = 10e6
    N = samp_steps
    # SCAM = Single Component Adaptive Metropolis
    # AM = Adaptive Metropolis
    # DE = Differential Evolution
    sampler.sample(x0, N, SCAMweight=30, AMweight=15, DEweight=50, )





